In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

tf.__version__

'2.10.0'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 당뇨병을 진단해봅시다.

In [2]:
xy = np.loadtxt('data-03-diabetes.csv',
                delimiter=',',
                dtype=np.float32)
x_train = xy[0:-100, 0:-1]
y_train = xy[0:-100, [-1]]

x_test = xy[-100:, 0:-1]
y_test = xy[-100:, [-1]]

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(659, 8) (659, 1)
(100, 8) (100, 1)


In [3]:
print(x_test[0])
print(y_test[0])

[-0.294118  -0.0150754 -0.0491803 -0.333333  -0.550827   0.0134128
 -0.699402  -0.266667 ]
[1.]


In [4]:
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(len(x_train))


Metal device set to: Apple M1


2023-12-19 14:41:05.621243: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-12-19 14:41:05.621813: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [ ]:
W = tf.Variable(tf.random.normal(# TODO), name='weight')
b = tf.Variable(tf.random.normal(# TODO), name='bias')


## 가설 설정
* 병이 있다 / 없다로 분류
* sigmoid classification으로 진행


## $$ y_k = \frac{1}{1+e^{-x}}   $$

In [ ]:
def logistic_regression(features):
    hypothesis  = # TODO
    return hypothesis

print(logistic_regression(x_train))

## Loss Function

* 기존 MSE 대신 Cross Entropy 사용

## $$
\begin{align}
loss(h(x),y) & = −y log(h(x))−(1−y)log(1−h(x))
\end{align}
$$

In [ ]:
def loss_fn(hypothesis, labels):
    cost = # TODO
    return cost

optimizer = # TODO

## 학습

In [ ]:
epochs = 5000

for step in range(epochs):
  for features, labels in dataset:
    with tf.GradientTape() as tape:
      loss_value = # TODO
      grads = # TODO
      optimizer.apply_gradients(grads_and_vars=zip(# TODO))
      if step % 100 == 0:
            print("Iter: {}, Loss: {:.4f}".format(step, loss_fn(logistic_regression(features),labels)))

## 테스트

In [ ]:
def accuracy_fn(hypothesis, labels):
    predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, labels), dtype=tf.float32))
    return accuracy

In [ ]:
test_acc = accuracy_fn(logistic_regression(x_test),y_test)
print("Testset Accuracy: {:.4f}".format(test_acc))